# `run` function, `RunResponse` class and event processing

In [1]:
# Chat between two comedian agents

# 1. Import our agent class
from autogen import ConversableAgent, LLMConfig
from autogen.io.run_response import RunResponseProtocol

# 2. Define our LLM configuration for OpenAI's GPT-4o mini,
#    uses the OPENAI_API_KEY environment variable
# llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")
llm_config = LLMConfig.from_json(path="OAI_CONFIG_LIST").where(model="gpt-4o-mini")
print(f"Using LLM: {llm_config}")

# 3. Create our agents who will tell each other jokes,
#    with Jack ending the chat when Emma says FINISH
with llm_config:
    jack = ConversableAgent(
        "Jack",
        system_message=("Your name is Jack and you are a comedian in a two-person comedy show."),
        is_termination_msg=lambda x: "FINISH" in x["content"],
        human_input_mode="NEVER",
    )
    emma = ConversableAgent(
        "Emma",
        system_message=(
            "Your name is Emma and you are a comedian "
            "in a two-person comedy show. Say the word FINISH "
            "ONLY AFTER you've heard 2 of Jack's jokes."
        ),
        human_input_mode="NEVER",
    )

# 4. Run the chat
response: RunResponseProtocol = jack.run(emma, message="Emma, tell me a joke about goldfish and peanut butter.")

for event in response.events:
    print(event)

    if event.type == "input_request":
        event.content.respond(input())

print(f"{response.summary=}")
print(f"{response.messages=}")

Using LLM: LLMConfig(config_list=[{'api_type': 'openai', 'model': 'gpt-4o-mini', 'api_key': '**********', 'tags': ['gpt-4o-mini', 'tool', 'vision']}, {'api_type': 'azure', 'model': 'gpt-4o-mini', 'api_key': '**********', 'api_version': '2024-05-01-preview', 'base_url': 'https://dev-openai-eastus-airt.openai.azure.com/', 'tags': ['gpt-4o-mini', 'tool', 'vision']}])
type='text' content=TextEvent(uuid=UUID('e4006fcb-f4cd-4511-a34f-76eee3ab8d88'), content='Emma, tell me a joke about goldfish and peanut butter.', sender_name='Jack', recipient_name='Emma')
type='text' content=TextEvent(uuid=UUID('8a40620a-c725-4839-9ef1-0be30e4f3116'), content='Sure, Jack! Why did the goldfish bring a jar of peanut butter to its aquarium? Because it wanted to make a little “fin”-tastic sandwich! How about you tell me a joke?', sender_name='Emma', recipient_name='Jack')
type='text' content=TextEvent(uuid=UUID('592c9b39-928c-4822-b17f-64d788660f7f'), content="Alright! Why did the scarecrow win an award? Becaus

# With console processor

In [2]:
# Chat between two comedian agents

# 1. Import console event processor

# 2. Create our agents who will tell each other jokes,
#    with Jack ending the chat when Emma says FINISH
with llm_config:
    jack = ConversableAgent(
        "Jack",
        system_message=("Your name is Jack and you are a comedian in a two-person comedy show."),
        is_termination_msg=lambda x: "FINISH" in x["content"],
        human_input_mode="NEVER",
    )
    emma = ConversableAgent(
        "Emma",
        system_message=(
            "Your name is Emma and you are a comedian "
            "in a two-person comedy show. Say the word FINISH "
            "ONLY AFTER you've heard 2 of Jack's jokes."
        ),
        human_input_mode="NEVER",
    )

# 3. Run the chat
response = jack.run(emma, message="Emma, tell me a joke about goldfish and peanut butter.")

response.process()

Jack (to Emma):

Emma, tell me a joke about goldfish and peanut butter.

--------------------------------------------------------------------------------
Emma (to Jack):

Sure, Jack! Why did the goldfish bring a jar of peanut butter to its aquarium? Because it wanted to make a little “fin”-tastic sandwich! How about you tell me a joke?

--------------------------------------------------------------------------------
Jack (to Emma):

Alright! Why did the scarecrow win an award? Because he was outstanding in his field! What's next on the joke menu, Emma?

--------------------------------------------------------------------------------
Emma (to Jack):

That’s a classic! Alright, here’s another one: Why don't scientists trust atoms? Because they make up everything! Your turn again, Jack!

--------------------------------------------------------------------------------
Jack (to Emma):

Okay, here we go! Why did the bicycle fall over? Because it was two-tired! Emma, what's the deal with bicy

In [3]:
# 1. Create our LLM agent
with llm_config:
    my_agent = ConversableAgent(
        name="helpful_agent",
        system_message="You are a poetic AI assistant, respond in rhyme.",
    )

# 2. Run the agent with a prompt
response = my_agent.run("In one sentence, what's the big deal about AI?")

response.process()

AttributeError: 'str' object has no attribute '_raise_exception_on_async_reply_functions'

In [4]:
# Group chat amongst agents to create a 4th grade lesson plan
# Flow determined by Group Chat Manager automatically, and
# should be Teacher > Planner > Reviewer > Teacher (repeats if necessary)

# 1. Import our agent and group chat classes
from autogen import GroupChat, GroupChatManager

with llm_config:
    # Planner agent setup
    planner_message = "Create lesson plans for 4th grade. Use format: <title>, <learning_objectives>, <script>"
    planner = ConversableAgent(name="planner_agent", system_message=planner_message, description="Creates lesson plans")

    # Reviewer agent setup
    reviewer_message = "Review lesson plans against 4th grade curriculum. Provide max 3 changes."
    reviewer = ConversableAgent(
        name="reviewer_agent", system_message=reviewer_message, description="Reviews lesson plans"
    )

    # Teacher agent setup
    teacher_message = "Choose topics and work with planner and reviewer. Say DONE! when finished."
    teacher = ConversableAgent(
        name="teacher_agent",
        system_message=teacher_message,
    )

# Setup group chat
groupchat = GroupChat(agents=[teacher, planner, reviewer], speaker_selection_method="auto", messages=[])

# Create manager
# At each turn, the manager will check if the message contains DONE! and end the chat if so
# Otherwise, it will select the next appropriate agent using its LLM
manager = GroupChatManager(
    name="group_manager",
    groupchat=groupchat,
    llm_config=llm_config,
    is_termination_msg=lambda x: "DONE!" in (x.get("content", "") or "").upper(),
)

# Start the conversation
response = teacher.run(recipient=manager, message="Let's teach the kids about the solar system.")

response.process()

teacher_agent (to group_manager):

Let's teach the kids about the solar system.

--------------------------------------------------------------------------------

Next speaker: planner_agent


>>>>>>>> USING AUTO REPLY...
planner_agent (to group_manager):

**Title:** Journey Through the Solar System

**Learning Objectives:**
1. Students will identify and describe the eight planets in our solar system.
2. Students will understand the concept of orbit and how planets differ in size and distance from the sun.
3. Students will recognize some characteristics of other celestial bodies, such as moons and asteroids.

**Script:**

**Introduction (10 minutes)**
- Teacher: "Good morning, class! Today, we are going to embark on a fantastic journey through our solar system. Who can tell me what the solar system is?"
- (Listen to student responses and guide them to understand the solar system is made up of the sun and all the objects that orbit it, including planets, moons, and asteroids.)
- Teacher

In [5]:
from autogen import AfterWorkOption, run_swarm

# 1. Create our agents
planner_message = """You are a classroom lesson planner.
Given a topic, write a lesson plan for a fourth grade class.
If you are given revision feedback, update your lesson plan and record it.
Use the following format:
<title>Lesson plan title</title>
<learning_objectives>Key learning objectives</learning_objectives>
<script>How to introduce the topic to the kids</script>
"""

reviewer_message = """You are a classroom lesson reviewer.
You compare the lesson plan to the fourth grade curriculum
and provide a maximum of 3 recommended changes for each review.
Make sure you provide recommendations each time the plan is updated.
"""

teacher_message = """You are a classroom teacher.
You decide topics for lessons and work with a lesson planner.
and reviewer to create and finalise lesson plans.
"""

with llm_config:
    lesson_planner = ConversableAgent(name="planner_agent", system_message=planner_message)

    lesson_reviewer = ConversableAgent(name="reviewer_agent", system_message=reviewer_message)

    teacher = ConversableAgent(
        name="teacher_agent",
        system_message=teacher_message,
    )

# 2. Initiate the swarm chat using a swarm manager who will
# select agents automatically
response = run_swarm(
    initial_agent=teacher,
    agents=[lesson_planner, lesson_reviewer, teacher],
    messages="Today, let's introduce our kids to the solar system.",
    max_rounds=10,
    swarm_manager_args={"llm_config": llm_config},
    after_work=AfterWorkOption.SWARM_MANAGER,
)

response.process()

_User (to chat_manager):

Today, let's introduce our kids to the solar system.

--------------------------------------------------------------------------------

Next speaker: teacher_agent


>>>>>>>> USING AUTO REPLY...
teacher_agent (to chat_manager):

That sounds like a fantastic topic for a lesson! The solar system is both fascinating and engaging for students. Here’s a draft lesson plan for introducing the solar system to your students:

### Lesson Plan: Introduction to the Solar System

**Grade Level:** 3rd-5th Grade  
**Duration:** 60 minutes  
**Objectives:**
- Students will be able to identify and name the planets in the solar system.
- Students will understand the basic characteristics of each planet.
- Students will learn about the sun and other celestial bodies within the solar system.

**Materials Needed:**
- Pictures or models of the solar system (including the sun, planets, and moons)
- A large poster or digital presentation of the solar system
- Craft supplies (construc